In [12]:
import pandas as pd

# RAIS

## Estabelecimentos


TODO

- Pipeline

    - Leitura
    - Tratamento
    - Merging
    - Load

ano, uf, meso, micro, muni

In [13]:
def read_data(caminho, ano)-> pd.DataFrame:
    '''Read the ESTB data from a file and return a DataFrame.'''
    try: # txt file
        colunas = ['CNAE 2.0 Classe', 'Município']
        df_estb = pd.read_csv(caminho + 'ESTB' + ano + '.txt', sep=';', encoding='latin1', usecols=colunas)
        df_estb['ano'] = ano
        extensao_arquivo = 'txt'
        
    except: # csv file
        colunas = ['ano', 'id_municipio', 'cnae_2']
        df_estb = pd.read_csv(caminho + 'ESTB' + ano + '.csv', usecols=colunas)
        extensao_arquivo = 'csv'

    return df_estb, extensao_arquivo

In [14]:
def transform_data(df, extensao) -> pd.DataFrame:
    '''Normaliza o tipo de cada coluna e obtem a cnae_secao de cnae_2'''
    
    if extensao == 'txt':
        df.rename(columns={'Município': 'id_municipio', 'CNAE 2.0 Classe': 'cnae_2'}, inplace=True)	
        df = df[['ano', 'id_municipio', 'cnae_2']]
    elif extensao == 'csv':
        df.dropna(inplace=True)
        df['id_municipio'] = df['id_municipio'].astype('int')
        df['id_municipio'] = df['id_municipio'].apply(lambda x: int(x/10)) # remove 7'th dígito
    
    # para todos
    df['ano'] = df['ano'].astype('int16')
    
    df['id_municipio'] = df['id_municipio'].astype('int')
    
    df.loc[:, 'cnae_2'] = df['cnae_2'].apply(lambda x: str(x).zfill(5))
    df['cnae_secao'] = df['cnae_2'].apply(lambda x: x[:2])
    df['cnae_secao'] = df['cnae_secao'].astype('int')
    df.drop(columns=['cnae_2'], inplace=True)
    return df

In [15]:
def merge_municipios(df, caminho, loc) -> pd.DataFrame:
    '''Faz o merge do id_municipio com os dados do dicionário'''
    
    colunas_muni = ['id_mesorregiao', 'id_microrregiao', 'id_municipio_6', 'nome', 'nome_microrregiao', 'nome_mesorregiao', 'sigla_uf', 'nome_uf', 'nome_regiao', 'centroide']
    dicionario_muni = pd.read_csv(caminho, sep=',', usecols=colunas_muni)
    dicionario_muni.rename(columns={'id_municipio_6': 'id_municipio', 'nome': 'municipio'}, inplace=True)
    
    df = pd.merge(df, dicionario_muni, on='id_municipio', how='left')
    return df

In [ ]:
def merge_cnae(df, caminho) -> pd.DataFrame:
    '''Faz o merge do cnae_secao com os dados do dicionário'''
    
    colunas_tmp = ['divisao','descricao_secao', 'descricao_divisao']
    df_cnae = pd.read_csv(caminho, usecols=colunas_tmp)
    df_cnae.rename(columns={'divisao': 'cnae_secao'}, inplace=True)
    df_cnae.drop_duplicates(inplace=True)   
    
    df = pd.merge(df, df_cnae, on='cnae_secao', how='left')
    df.drop(columns=['cnae_secao'], inplace=True)
    return df

In [20]:
def calc_ql(df, loc='meso') -> pd.DataFrame:
    '''
    Calcula o quociente de localização para níveis de meso, microrregião ou município tendo o estado como setor de referência
    loc= 'meso', 'micro' ou 'muni'
    '''
    
    if loc == 'meso':
        numerador = df.groupby(['ano', 'sigla_uf', 'nome_mesorregiao', 'id_mesorregiao', 'descricao_secao']).size() / df.groupby(['ano', 'sigla_uf', 'nome_mesorregiao', 'id_mesorregiao']).size()
        denominador = df.groupby(['ano', 'sigla_uf', 'descricao_secao']).size() / df.groupby(['ano', 'sigla_uf']).size()
        ql = numerador / denominador
        ql = ql.reset_index()
        ql.columns = ['ano', 'sigla_uf', 'nome_mesorregiao', 'id_mesorregiao', 'descricao_secao', 'quociente_localizacao']
        
    elif loc == 'micro':
        numerador = df.groupby(['ano', 'sigla_uf', 'nome_mesorregiao', 'nome_microrregiao', 'id_microrregiao', 'descricao_secao']).size() / df.groupby(['ano', 'sigla_uf', 'nome_mesorregiao', 'nome_microrregiao', 'id_microrregiao']).size()
        denominador = df.groupby(['ano', 'sigla_uf', 'descricao_secao']).size() / df.groupby(['ano', 'sigla_uf']).size()
        ql = numerador / denominador
        ql = ql.reset_index()
        ql.columns = ['ano', 'sigla_uf', 'nome_mesorregiao', 'nome_microrregiao', 'id_microrregiao', 'descricao_secao', 'quociente_localizacao']
        
    elif loc == 'muni':
        numerador = df.groupby(['ano', 'sigla_uf', 'nome_mesorregiao', 'nome_microrregiao', 'id_municipio', 'municipio', 'centroide', 'descricao_secao']).size() / df.groupby(['ano', 'sigla_uf', 'nome_mesorregiao', 'nome_microrregiao', 'id_municipio', 'municipio', 'centroide']).size()
        denominador = df.groupby(['ano', 'sigla_uf', 'descricao_secao']).size() / df.groupby(['ano', 'sigla_uf']).size()
        ql = numerador / denominador
        ql = ql.reset_index()
        ql.columns = ['ano', 'sigla_uf', 'nome_mesorregiao', 'nome_microrregiao', 'id_municipio', 'municipio', 'centroide', 'descricao_secao', 'quociente_localizacao']

    ql['quociente_localizacao'] = ql['quociente_localizacao'].map(lambda x: float(f"{x:.2f}"))

    return ql


In [21]:
def etl_pipeline(caminhos, ano_ini, ano_fim, loc):
    '''
    Executa o pipeline ETL para os dados de estabelecimentos da RAIS.

    Parâmetros:
        caminhos (dict): Dicionário com os caminhos dos arquivos de dados e dicionários.
        ano_ini (int): Ano inicial do processamento.
        ano_fim (int): Ano final do processamento.
        loc (str): Nível de localização para o cálculo do quociente de localização ('meso', 'micro' ou 'muni').

    Retorna:
        pd.DataFrame: DataFrame com o quociente de localização calculado para o(s) ano(s) e nível selecionados.
    '''
    dfs_ql = []
    for ano in range(ano_ini, ano_fim+1):
        print('---', ano, '---')
        df, extensao = read_data(caminhos['rais'], str(ano))
        df = transform_data(df, extensao)
        df = merge_municipios(df, caminhos['municipios'], loc)
        df = merge_cnae(df, caminhos['cnae'])
        
        dfs_ql.append(calc_ql(df, loc))
    df = pd.concat(dfs_ql)
    return df

---

In [ ]:
from pathlib import Path

caminho = "D:/dados/rais/estabelecimentos/"
caminho_dict_muni = "dicionarios/dicionario_municipios.csv"
caminho_dict_cnae = "dicionarios/dicionario_cnae_2.csv"
ano_ini = 2007
ano_fim = 2024
caminhos = {'rais': caminho, 
            'municipios': caminho_dict_muni, 
            'cnae': caminho_dict_cnae}

for loc in ['muni', 'micro', 'meso']:
    df_ql = etl_pipeline(caminhos, ano_ini, ano_fim, loc)
    df_ql.to_csv(Path('indices_ql/indice_ql_' + loc + '.csv'), sep=';', index=False)
    print('---', loc, '---')

--- 2007 ---


C:\Users\ramos\AppData\Local\Temp\ipykernel_8152\2608428684.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['64221' '64221' '64221' ... '71111' '81125' '94995']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[:, 'cnae_2'] = df['cnae_2'].apply(lambda x: str(x).zfill(5))


--- 2008 ---


C:\Users\ramos\AppData\Local\Temp\ipykernel_8152\1388004525.py:5: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_estb = pd.read_csv(caminho + 'ESTB' + ano + '.txt', sep=';', encoding='latin1', usecols=colunas)


--- 2009 ---


...

--- 2023 ---


C:\Users\ramos\AppData\Local\Temp\ipykernel_8152\2608428684.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['47890' '42995' '94308' ... '47822' '43118' '11135']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[:, 'cnae_2'] = df['cnae_2'].apply(lambda x: str(x).zfill(5))


--- 2024 ---


C:\Users\ramos\AppData\Local\Temp\ipykernel_8152\2608428684.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['47725' '69117' '47121' ... '03124' '03124' '41204']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[:, 'cnae_2'] = df['cnae_2'].apply(lambda x: str(x).zfill(5))


--- muni ---


---